<a href="https://colab.research.google.com/github/E-xxi/kaggle_house-price/blob/main/houseprice%5B2%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# House Prices - Advanced Regression Techniques
sale prices를 예측

### 설정

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

%cd /content/gdrive/My Drive/Kaggle
%pwd


Mounted at /content/gdrive
/content/gdrive/My Drive/Kaggle


'/content/gdrive/My Drive/Kaggle'

In [ ]:
# 데이터 다운받기
!kaggle competitions download -c house-prices-advanced-regression-techniques
!ls

%mkdir input/house-price
%mv house-prices-advanced-regression-techniques.zip input/house-price
%cd input/house-price/

!unzip \*.zip  && rm *.zip

### 연결

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/content/gdrive/My Drive/Kaggle/input/house-price'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

%pwd
%cd input/house-price/

data = pd.read_csv('train.csv')
OriginData = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

/content/gdrive/My Drive/Kaggle/input/house-price/test.csv
/content/gdrive/My Drive/Kaggle/input/house-price/train.csv
/content/gdrive/My Drive/Kaggle/input/house-price/data_description.txt
/content/gdrive/My Drive/Kaggle/input/house-price/sample_submission.csv
/content/gdrive/My Drive/Kaggle/input/house-price/result1.csv
/content/gdrive/My Drive/Kaggle/input/house-price/result2.csv
/content/gdrive/My Drive/Kaggle/input/house-price




---
## 1. control missing values : 0.15209


In [ ]:
#feature와 target 설정
from sklearn.model_selection import train_test_split
#train할수없는 SalePrice가 NaN인 경우 행삭제
data.dropna(axis=0, subset=['SalePrice'], inplace = True)

y = data.SalePrice
data.drop(['SalePrice'],axis=1, inplace = True)
X = data.select_dtypes(exclude=['object']) #numerical만 다루도록한다. 

X_test = test.select_dtypes(exclude=['object']) #test용 feature도 numerical만 

In [ ]:
#우선적으로 결측값 처리를 한다. 
threshold = 0.7

#Dropping columns with missing value rate higher than threshold
data = data[data.columns[data.isnull().mean() < threshold]]
#Dropping rows with missing value rate higher than threshold
data = data.loc[data.isnull().mean(axis=1) < threshold]

결측값 처리가 필요한 열을 파악한다. 

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 37)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


LotFrontage, MasVnrArea, GarageYrBlt에서 결측값을 확인할 수 있다. 

- LotFrontage, MasVnrArea, GarageYrBlt:  관련한 다른 feature들에는 값이 존재하므로 extansion이 필요하진 않다. 


---



### imputation을 이용하여 결측값 처리







In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [ ]:
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

19243.57294520548


처리한 데이터 전체를 넣어서 모델을 확인한다.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
estimat_num = [10,100,200,500,1000]
for e in estimat_num:
  choosing_final_model = RandomForestRegressor(n_estimators = e, min_samples_split = 20)
  choosing_final_model.fit(imputed_X_train, y_train)
  preds = choosing_final_model.predict(imputed_X_valid)
  print(mean_absolute_error(y_valid, preds))

best_esti = 200

19249.771337427366
18575.617623673406
18463.079706955625
18367.530847545782
18515.918228043913


In [ ]:
split_num = [10,100,200,500,1000]
for e in estimat_num:
  choosing_final_model = RandomForestRegressor(n_estimators = best_esti, min_samples_split = e)
  choosing_final_model.fit(imputed_X_train, y_train)
  preds = choosing_final_model.predict(imputed_X_valid)
  print(mean_absolute_error(y_valid, preds))
best_split = 10

18167.567092837933
23634.45953471177
28961.41019938502
34756.79785886537
57062.02020635906


In [ ]:
imputer = SimpleImputer()
imputed_X = pd.DataFrame(imputer.fit_transform(X))

final_model = RandomForestRegressor(n_estimators = best_esti, min_samples_split = best_split)
final_model.fit(imputed_X, y)

RandomForestRegressor(min_samples_split=10, n_estimators=200)

In [ ]:
imputed_X_test = pd.DataFrame(imputer.fit_transform(X_test))

# Generate test predictions
preds_test = final_model.predict(imputed_X_test)

# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.Id,
                       'SalePrice': preds_test})
output.to_csv('result3.csv', index=False)